In [1]:
import os
import subprocess
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

# ---------- Configuration ----------
MAX_WORKERS = 1
EXCLUDE_PREFIXES = [
    '__pycache__', 
    '.ipynb_checkpoints', 
    'base', 
    'completed runs', 
    'to do', 
    'analysis'
]

# Lock for thread-safe printing
print_lock = Lock()

# ---------- Core Helpers ----------
def run_notebook(folder: str, notebook_name: str) -> str:
    """
    Execute a Jupyter notebook in place using nbconvert.
    Returns a status string with execution time.
    All printing inside this function is thread-safe.
    """
    notebook_path = os.path.join(folder, notebook_name)
    if not os.path.exists(notebook_path):
        with print_lock:
            print(f"SKIP: {notebook_name} not found in {folder}", flush=True)
        return f"SKIP: {notebook_name} not found in {folder}"

    with print_lock:
        print(f"🔹 Running {notebook_path}...", flush=True)

    start_time = time.time()
    try:
        subprocess.run(
            [
                'jupyter', 'nbconvert', '--to', 'notebook',
                '--execute', notebook_path, '--inplace',
                '--ExecutePreprocessor.timeout=-1'
            ],
            check=True,
            capture_output=True,
            text=True
        )
        elapsed = time.time() - start_time
        return f"✅ SUCCESS: {folder}/{notebook_name} (took {elapsed:.2f}s)"
    except subprocess.CalledProcessError as e:
        elapsed = time.time() - start_time
        stderr_clean = e.stderr.strip()
        return f"❌ ERROR: {folder}/{notebook_name} - {stderr_clean} (took {elapsed:.2f}s)"

def find_folders(base_dir: str = '.') -> list[str]:
    """Return a list of folders to process, excluding certain prefixes."""
    return [
        d for d in os.listdir(base_dir)
        if os.path.isdir(d) and not any(d.startswith(prefix) for prefix in EXCLUDE_PREFIXES)
    ]

def run_in_parallel(folders: list[str], notebook_names: list[str]):
    """Run one or more notebooks across all folders in parallel, with thread-safe printing."""
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [
            executor.submit(run_notebook, folder, nb)
            for folder in folders
            for nb in notebook_names
        ]

        for future in as_completed(futures):
            msg = future.result().strip()
            with print_lock:
                print(msg)
    with print_lock:
        print("✅ All done!")

# ---------- Specialized Variants ----------
def run_random_perturbs_parallel():
    """Run Random Perturbs.ipynb across all model_* folders."""
    folders = find_folders()
    with print_lock:
        print(f"Found {len(folders)} folders for Random Perturbs.")
    run_in_parallel(folders, ["Random Perturbs.ipynb"])

def run_volume_parallel():
    """Run all notebooks containing 'Volume Cutoff' or 'Volume Estimation' across folders."""
    folders = find_folders()
    with print_lock:
        print(f"Found {len(folders)} folders for Volume notebooks.")

    notebook_names = []
    if folders:
        for nb in os.listdir(folders[0]):
            if nb.endswith(".ipynb") and ("Volume Cutoff" in nb or "Volume Estimation" in nb):
                notebook_names.append(nb)

    if not notebook_names:
        with print_lock:
            print("⚠️ No Volume notebooks found in reference folder.")
        return

    with print_lock:
        print(f"Will run these notebooks: {notebook_names}")
    run_in_parallel(folders, notebook_names)

def run_nb_parallel():
    """Generic function for ad-hoc notebook runs (prompt user)."""
    notebook_name = input("Enter notebook name (without .ipynb): ").strip() + ".ipynb"
    folders = find_folders()
    with print_lock:
        print(f"Found {len(folders)} folders to process.")
    run_in_parallel(folders, [notebook_name])

run_volume_parallel()

Found 10 folders for Volume notebooks.
Will run these notebooks: ['Volume Cutoff.ipynb', 'Volume Estimation Pipeline.ipynb']
🔹 Running model_0_data_10\Volume Cutoff.ipynb...
✅ SUCCESS: model_0_data_10/Volume Cutoff.ipynb (took 41.09s)
🔹 Running model_0_data_10\Volume Estimation Pipeline.ipynb...
✅ SUCCESS: model_0_data_10/Volume Estimation Pipeline.ipynb (took 67.17s)
🔹 Running model_1_data_11\Volume Cutoff.ipynb...
✅ SUCCESS: model_1_data_11/Volume Cutoff.ipynb (took 39.07s)
🔹 Running model_1_data_11\Volume Estimation Pipeline.ipynb...
✅ SUCCESS: model_1_data_11/Volume Estimation Pipeline.ipynb (took 67.29s)
🔹 Running model_2_data_12\Volume Cutoff.ipynb...
✅ SUCCESS: model_2_data_12/Volume Cutoff.ipynb (took 39.07s)
🔹 Running model_2_data_12\Volume Estimation Pipeline.ipynb...
✅ SUCCESS: model_2_data_12/Volume Estimation Pipeline.ipynb (took 67.84s)
🔹 Running model_3_data_13\Volume Cutoff.ipynb...
✅ SUCCESS: model_3_data_13/Volume Cutoff.ipynb (took 39.47s)
🔹 Running model_3_data_13\V

❌ ERROR: model_0_data_10/Volume Cutoff.ipynb - [NbConvertApp] Converting notebook model_0_data_10\Volume Cutoff.ipynb to notebook
L:\Programming\diffusion-env\Lib\site-packages\zmq\_future.py:724: RuntimeWarning: Proactor event loop does not implement add_reader family of methods required for zmq. Registering an additional selector thread for add_reader support via tornado. Use `asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())` to avoid this warning.
  self._get_loop()
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "L:\Programming\diffusion-env\Scripts\jupyter-nbconvert.EXE\__main__.py", line 7, in <module>
  File "L:\Programming\diffusion-env\Lib\site-packages\jupyter_core\application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "L:\Programming\diffusion-env\Lib\site-packages\traitlets\config\application.py", line 1075, in laun

🔹 Running model_1_data_11\Volume Cutoff.ipynb...


✅ SUCCESS: model_0_data_10/Volume Estimation Pipeline.ipynb (took 57.15s)


❌ ERROR: model_1_data_11/Volume Cutoff.ipynb - [NbConvertApp] Converting notebook model_1_data_11\Volume Cutoff.ipynb to notebook
L:\Programming\diffusion-env\Lib\site-packages\zmq\_future.py:724: RuntimeWarning: Proactor event loop does not implement add_reader family of methods required for zmq. Registering an additional selector thread for add_reader support via tornado. Use `asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())` to avoid this warning.
  self._get_loop()
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "L:\Programming\diffusion-env\Scripts\jupyter-nbconvert.EXE\__main__.py", line 7, in <module>
  File "L:\Programming\diffusion-env\Lib\site-packages\jupyter_core\application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "L:\Programming\diffusion-env\Lib\site-packages\traitlets\config\application.py", line 1075, in laun

...

✅ SUCCESS: model_7_data_17/Volume Estimation Pipeline.ipynb (took 57.69s)
🔹 Running model_8_data_18\Volume Cutoff.ipynb...


❌ ERROR: model_8_data_18/Volume Cutoff.ipynb - [NbConvertApp] Converting notebook model_8_data_18\Volume Cutoff.ipynb to notebook
L:\Programming\diffusion-env\Lib\site-packages\zmq\_future.py:724: RuntimeWarning: Proactor event loop does not implement add_reader family of methods required for zmq. Registering an additional selector thread for add_reader support via tornado. Use `asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())` to avoid this warning.
  self._get_loop()
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "L:\Programming\diffusion-env\Scripts\jupyter-nbconvert.EXE\__main__.py", line 7, in <module>
  File "L:\Programming\diffusion-env\Lib\site-packages\jupyter_core\application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "L:\Programming\diffusion-env\Lib\site-packages\traitlets\config\application.py", line 1075, in laun

✅ SUCCESS: model_8_data_18/Volume Estimation Pipeline.ipynb (took 57.43s)
🔹 Running model_9_data_19\Volume Cutoff.ipynb...


❌ ERROR: model_9_data_19/Volume Cutoff.ipynb - [NbConvertApp] Converting notebook model_9_data_19\Volume Cutoff.ipynb to notebook
L:\Programming\diffusion-env\Lib\site-packages\zmq\_future.py:724: RuntimeWarning: Proactor event loop does not implement add_reader family of methods required for zmq. Registering an additional selector thread for add_reader support via tornado. Use `asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())` to avoid this warning.
  self._get_loop()
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "L:\Programming\diffusion-env\Scripts\jupyter-nbconvert.EXE\__main__.py", line 7, in <module>
  File "L:\Programming\diffusion-env\Lib\site-packages\jupyter_core\application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "L:\Programming\diffusion-env\Lib\site-packages\traitlets\config\application.py", line 1075, in laun

✅ SUCCESS: model_9_data_19/Volume Estimation Pipeline.ipynb (took 58.37s)
✅ All done!
